In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler


def normalize(data):
    
    normalizer = StandardScaler().fit(data)
    
    return normalizer.transform(data), normalizer

x = np.array([
    [100, 0.1, -4],
    [200, 0.3, -9],
    [500, 0.2, 0]
])
x_normed, x_normalizer = normalize(x)

x_normed

array([[-9.80580676e-01, -1.22474487e+00,  9.05357460e-02],
       [-3.92232270e-01,  1.22474487e+00, -1.26750044e+00],
       [ 1.37281295e+00, -3.39934989e-16,  1.17696470e+00]])

In [4]:
lenni = 3987
tr_re_va_te = np.array([0.4, 0.4, 0.1, 0.1])

N_train, N_recal, N_val, N_test = (lenni * tr_re_va_te).astype(int)
N_train                      

1594

In [8]:
import torch

Y = torch.Tensor([1, 9, 4, 2])

upp = torch.Tensor([3,3,3,3])
low = torch.Tensor([1,1,1,1])

((Y < upp) & (Y > low)).float().sum()

tensor(1.)

In [1]:
import numpy as np

np.linspace(0.01, 0.99, 100)

array([0.01      , 0.01989899, 0.02979798, 0.03969697, 0.04959596,
       0.05949495, 0.06939394, 0.07929293, 0.08919192, 0.09909091,
       0.1089899 , 0.11888889, 0.12878788, 0.13868687, 0.14858586,
       0.15848485, 0.16838384, 0.17828283, 0.18818182, 0.19808081,
       0.2079798 , 0.21787879, 0.22777778, 0.23767677, 0.24757576,
       0.25747475, 0.26737374, 0.27727273, 0.28717172, 0.29707071,
       0.3069697 , 0.31686869, 0.32676768, 0.33666667, 0.34656566,
       0.35646465, 0.36636364, 0.37626263, 0.38616162, 0.39606061,
       0.4059596 , 0.41585859, 0.42575758, 0.43565657, 0.44555556,
       0.45545455, 0.46535354, 0.47525253, 0.48515152, 0.49505051,
       0.50494949, 0.51484848, 0.52474747, 0.53464646, 0.54454545,
       0.55444444, 0.56434343, 0.57424242, 0.58414141, 0.5940404 ,
       0.60393939, 0.61383838, 0.62373737, 0.63363636, 0.64353535,
       0.65343434, 0.66333333, 0.67323232, 0.68313131, 0.6930303 ,
       0.70292929, 0.71282828, 0.72272727, 0.73262626, 0.74252